In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-file/tested.csv


In [2]:
df = pd.read_csv('/kaggle/input/test-file/tested.csv')[['Age', 'Pclass', 'SibSp','Parch','Survived']]

In [3]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,34.5,3,0,0,0
1,47.0,3,1,0,1
2,62.0,2,0,0,0
3,27.0,3,0,0,0
4,22.0,3,1,1,1


* Here, Survived is our output column.
* First, we will train our model normally without any construction or splitting


In [4]:
df.dropna(inplace= True)

In [6]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,34.5,3,0,0,0
1,47.0,3,1,0,1
2,62.0,2,0,0,0
3,27.0,3,0,0,0
4,22.0,3,1,1,1


In [7]:
X = df.iloc[:, 0:4]
y = df.iloc[:, -1]

In [8]:
X.head()

,Age,Pclass,SibSp,Parch
0,34.5,3,0,0
1,47.0,3,1,0
2,62.0,2,0,0
3,27.0,3,0,0
4,22.0,3,1,1


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns

In [11]:
np.mean(cross_val_score(LogisticRegression(),X,y,scoring='accuracy',cv=20))

0.6238970588235293

# **Applying Feature Construction**

In [12]:
X['Family_size'] = X['SibSp'] + X['Parch'] +1

In [13]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size
0,34.5,3,0,0,1
1,47.0,3,1,0,2
2,62.0,2,0,0,1
3,27.0,3,0,0,1
4,22.0,3,1,1,3


In [14]:
def myfunc(num):
    if num==1:
        #alone
        return 0
    elif num>1 and num <=4:
        #small family
        return 1
    else:
        #large family
        return 2
    

In [15]:
myfunc(4)

1

In [16]:
X['Family_type'] = X['Family_size'].apply(myfunc)

In [17]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size,Family_type
0,34.5,3,0,0,1,0
1,47.0,3,1,0,2,1
2,62.0,2,0,0,1,0
3,27.0,3,0,0,1,0
4,22.0,3,1,1,3,1


In [18]:
X.drop(columns=['SibSp','Parch','Family_size'], inplace = True)

In [19]:
X.head()

,Age,Pclass,Family_type
0,34.5,3,0
1,47.0,3,1
2,62.0,2,0
3,27.0,3,0
4,22.0,3,1


In [22]:
np.mean(cross_val_score(LogisticRegression(), X,y, scoring='accuracy', cv=5))

0.617277250113071

# **Feature Splitting**

In [23]:
dff = pd.read_csv('/kaggle/input/test-file/tested.csv')

In [24]:
dff.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
dff['Name']

0                                  Kelly, Mr. James
1                  Wilkes, Mrs. James (Ellen Needs)
2                         Myles, Mr. Thomas Francis
3                                  Wirz, Mr. Albert
4      Hirvonen, Mrs. Alexander (Helga E Lindqvist)
                           ...                     
413                              Spector, Mr. Woolf
414                    Oliva y Ocana, Dona. Fermina
415                    Saether, Mr. Simon Sivertsen
416                             Ware, Mr. Frederick
417                        Peter, Master. Michael J
Name: Name, Length: 418, dtype: object

In [28]:
# the below code is used to extract the title/ salutation of the name column

dff['Title'] = dff['Name'].str.split(', ', expand =True)[1].str.split('.', expand=True)[0]

In [29]:
dff['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

0          Mr
1         Mrs
2          Mr
3          Mr
4         Mrs
        ...  
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: 0, Length: 418, dtype: object

In [31]:
dff[['Title' , 'Name']]

,Title,Name
0,Mr,"Kelly, Mr. James"
1,Mrs,"Wilkes, Mrs. James (Ellen Needs)"
2,Mr,"Myles, Mr. Thomas Francis"
3,Mr,"Wirz, Mr. Albert"
4,Mrs,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)"
...,...,...
413,Mr,"Spector, Mr. Woolf"
414,Dona,"Oliva y Ocana, Dona. Fermina"
415,Mr,"Saether, Mr. Simon Sivertsen"
416,Mr,"Ware, Mr. Frederick"


In [34]:
dff['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Col', 'Rev', 'Dr', 'Dona'],
      dtype=object)

In [33]:
(dff.groupby('Title').mean()['Survived']).sort_values(ascending = False)

/tmp/ipykernel_32/2808230348.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  (dff.groupby('Title').mean()['Survived']).sort_values(ascending = False)


Title
Dona      1.0
Miss      1.0
Mrs       1.0
Ms        1.0
Col       0.0
Dr        0.0
Master    0.0
Mr        0.0
Rev       0.0
Name: Survived, dtype: float64

In [36]:
dff['Is_Married'] =0
dff['Is_Married'].loc[dff['Title']=='Mrs'] =1

/tmp/ipykernel_32/3217601581.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Is_Married'].loc[dff['Title']=='Mrs'] =1


In [37]:
dff['Is_Married']

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    0
415    0
416    0
417    0
Name: Is_Married, Length: 418, dtype: int64